Author: Judit Lozano Gondolbeu

### Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, ConfusionMatrixDisplay, roc_curve, precision_recall_curve,\
                             classification_report,  accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
import scikitplot as skplt
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.datasets import make_hastie_10_2
from sklearn.inspection import PartialDependenceDisplay
import shap
import lightgbm as lgbm


warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)


In [2]:
with open('../data/train_ridge.pickle', 'rb') as f:
     X_train, y_train, X_test, y_test = pickle.load(f)
      # display dataset
print(X_train.head())

     C_MNTH    C_WDAY    C_HOUR    C_RCFG    C_WTHR    C_RALN    C_TRAF  \
0 -1.129054  1.375704  0.651298 -0.824320  0.948994  0.531235 -0.363466   
1 -1.129054 -0.417392 -0.779007 -0.824320 -0.246324 -0.353054 -1.353824   
2  0.183556 -0.417392 -0.646253  1.246626 -0.246324 -0.353054 -0.318034   
3  1.797639 -0.899118 -0.359585  1.246626 -0.246324 -0.353054  0.851024   
4 -1.413490 -0.504111 -0.556747 -0.605457  0.909085 -1.210185 -1.353824   

     V_TYPE    P_SAFE    V_YEAR     P_AGE     P_SEX    PP_CAR  
0 -0.251504 -0.225190 -0.118230 -0.963827  0.816670 -1.450629  
1 -0.251504 -0.225190  0.097020 -0.591176 -1.219878  0.839152  
2 -0.251504  0.074408 -0.154105 -0.692808 -1.219878  0.839152  
3 -0.711926 -0.369937  3.433395 -0.828318 -1.219878  0.174065  
4 -0.251504 -0.225190 -0.261729 -0.658931 -1.219878  0.839152  


In [3]:
#Cargamos el modelo entrenado y optimizado
with open("../models/model_lgb_final.pckl", "rb") as f:
    model = pickle.load(f)

In [4]:
parametros= model.best_params_
print(parametros)

{'class_weight': {0: 0.2, 1: 0.8}, 'learning_rate': 0.01, 'n_estimators': 1000, 'num_leaves': 50}


In [5]:
lgb_model = lgbm.LGBMClassifier(**parametros)

In [6]:
lgb_model.fit(X_train, y_train)

LGBMClassifier(class_weight={0: 0.2, 1: 0.8}, learning_rate=0.01,
               n_estimators=1000, num_leaves=50)

### Interpretabilidad Local con SHAP

Con Shap podemos estudiar la correlación ente cada variable con la variable target para uno o varias filas y ordena las variables segun sus coeficientes absolutos de mayor a menor.

In [ ]:
# Create object that can calculate shap values
explainer = shap.TreeExplainer(lgb_model)

# Calculate Shap values
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], X_test.iloc[0,:])

In [ ]:
#summarizetheeffectsofallthefeatures
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values[0],features=X_test)

Si analizamos los valores para una única fila el punto donde se diferencia el color rojo del azul es la probabilidad de que ese determinado caso la variable target sea un uno, es decir probabilidad de que ese conductor esté involucrado en un accidente donde habrá algun fallecido. 

Las variables en azul hacen que esa probabilidad baje y las variables en rojo hacen que la probabilidad suba.

### Conclusiones

En este estudio aplicamos herramientas de machine learning para un problema de clasificación donde queremos predecir si habrá o no fatalidades dado un accidente entre los conductores asegurados por las empresas aseguradoras de automóviles con cobertura en Canadá. 

Tras realizar un análisis exploratorio inicial del data set observamos que este está muy desbalanceado y que no hay correlaciones destacables entre las variables que son mayoritariamente de naturaleza categórica.

Para atajar el problema de desbalanceo de datos y tras un procesado y limpieza de las variables del dataset hemos aplicado la técnica de SMOTE combinada con un ligero undersampling para facilitar el entrenado posterior del modelo.

Tras estudiar las métricas con varios modelos, optimización de hiperparámetros y validación cruzada nuestro modelo ganador es el LGBMClassifier. 

En este estudio nos hemos centrado en la información a nivel global ya que nos interesa conocer el número global de conductores involucrados en accidentes con resultado mortal con la intención de estimar las reservas necesarias en caso de indemnización. 

Para un estudio a nivel local donde los objetivos sean el abordaje del cálculo de la prima por ejemplo para un nuevo conductor que quiere asegurarse con la compañia, tendríamos que replantear el estudio completamente dando especial importancia a la parte de interpretabilidad a nivel local para evitar sobretodo posibles sesgos descriminatorios de carácter demográfico. 


